In [1]:
import pandas as pd
import numpy as np
import codecs
import sys
import nltk
from bs4 import BeautifulSoup
import re
import gensim
import string

import multiprocessing as mp
import time

from IPython.display import clear_output

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk import word_tokenize
import spacy
#import emot
#from emot.emo_unicode import EMOTICONS_EMO
from spellchecker import SpellChecker
import swifter
import re
import dask.dataframe as dd
#from translate import Translator
#import translators as ts
#import translators.server as tss
#from emot.emo_unicode import UNICODE_EMO, EMOTICONS

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\e11925939\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\e11925939\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#doc = codecs.open('../data/coronaPostings.csv','rU','UTF-8') #open for reading with "universal" type set
doc = codecs.open('../../data/coronaPostings1.csv','rU','UTF-16')
sRows = [248882, 302514, 432776, 539306, 722492, 926415, 1003956, 1113394]
data = pd.read_csv(doc, sep='\t', skiprows=sRows)
print(data.shape)
data.head()

C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_36720\3874686528.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(doc, sep='\t', skiprows=sRows)


(981153, 10)


,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive
0,1051800287,NaN,2000116189812,184470,1.0,Nicht sinnvoll,"Forschungen, Studien, Statistiken unddieganze ...",2020-03-26 16:09:27.953,True,False
1,1051801262,NaN,2000116189812,240940,1.0,super,erst ganz unten steht das wichtigste,NaN,NaN,NaN
2,hält von den Schlussfolgerungen aus den Simula...,2020-03-26 16:28:10.400,True,False,NaN,NaN,NaN,NaN,NaN,NaN
3,1054666961,1054666937,2000117671399,602300,1.0,NaN,es handelt sich um 1 Pressekonferenz. Mit Paus...,2020-05-25 09:19:27.010,False,True
4,1054667707,NaN,2000117671399,56623,1.0,NaN,die DB app sollte vor allem eines: gut funktio...,NaN,NaN,NaN


In [3]:
##drop missformated rows where values are NaN (excluding ID_Posting_Parent)
df = data.copy()
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.dropna(thresh=8)
df = df.dropna(subset=["Comment", "createdAt", "ID_GodotObject"])
df.createdAt = df.createdAt.fillna("")
df["ID_Posting_Parent"] = df["ID_Posting_Parent"].fillna(-1)
time_regex = "^(\d{4}-\d{2}-\d{2})"
df = df[df.createdAt.str.match(time_regex)]
df = df[df.ID_Posting.str.isdigit()]
df = df[df.ID_GodotObject.str.isdigit()]

df['createdAt'] = pd.to_datetime(df['createdAt'])
df.ID_GodotObject = df.ID_GodotObject.astype('int64')
df.ID_Posting = df.ID_Posting.astype('int64')
#
df.ID_Posting_Parent = df.ID_Posting_Parent.astype('int64')
df = df.sort_values(by='createdAt').reset_index(drop=True)
df = df.drop_duplicates(subset=["ID_Posting"])
print(df.shape)
print(df.dtypes)
df.head()

(421572, 10)
ID_Posting                       int64
ID_Posting_Parent                int64
ID_GodotObject                   int64
ID_CommunityIdentity            object
id_posting_status               object
Headline                        object
Comment                         object
createdAt               datetime64[ns]
VoteNegative                    object
VotePositive                    object
dtype: object


,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive
0,1049737336,-1,2000114365338,568012,1.0,NaN,Das Virus ist wohl doch VIEL ansteckender als ...,2020-02-10 08:48:31.513,False,True
2,1049737620,1049737336,2000114365338,505220,1.0,NaN,"So ist es, somit sind bestimmt auch deutlich m...",2020-02-10 08:59:34.357,False,True
4,1049737708,-1,2000114365338,508661,1,NaN,Das ist für mich eine Horrorvorstellung. Final...,2020-02-10 09:03:20.527,False,True
6,1049737722,-1,2000114365338,583906,1,NaN,"Wäre interessant zu wissen, ob vor allem schon...",2020-02-10 09:03:48.857,True,False
8,1049737801,1049737746,2000114365338,635467,1,NaN,Besser als Häfn! ;-),2020-02-10 09:07:10.360,False,True


In [4]:
##find the maximal deph in the forumnsposting

def find_maximum_depth(posting_id, depth):
    # find all direct replies for the posting
    #print(posting_id)
    replies = df[df["ID_Posting_Parent"] == posting_id]
    #print(len(replies))
    if replies.empty:
        # Return the current depth if there are no replies
        return depth
    else:
        current_depth = depth
        for row in replies.itertuples():
            current_depth = max(current_depth, find_maximum_depth(row.ID_Posting, depth+1))
        return current_depth
    
    
def delete_after_depth(posting_id, depth, data, delete_list = []):
    # find all direct replies for the posting
    #print(posting_id)
    replies = data[data["ID_Posting_Parent"] == posting_id]
    #print(len(replies))
    if replies.empty:
        # Return the current depth if there are no replies
        return delete_list
    else:
        #depth -= 1
        for row in replies.itertuples():
            delete_list.append(row.ID_Posting)
            delete_list += delete_after_depth(row.ID_Posting, depth-1, data)
        return delete_list
    
    
# Converting emoticons to words and track emojis that were replaced
emojis = [':D', ':3', '%)', ':(', ';D', '8-)', 'XD', ':-)', ':*', ':)', 'd:', ':O', ':P', ':/', ':-))', ":')", '=L', ';)', ':-*', ';_;', ':o', ':-(', '=D']
EMOTICONS_DE = {
    u":D":"Lachen",
    u":3":"glückliches Gesicht",
    u"%)":"betrunken und verwirrt",
    u":(":"traurig",
    u";D":"zwinkernd lachend",
    u"8-)":"glücklich",
    u"XD":"witzig",
    u":-)":"zufrieden",
    u":*":"Kuss",
    u":)":"zufrieden",
    u"d:":"spielend",
    u":O":"schockiert",
    u":P":"spielend",
    u":/":"mittelmäßig",
    u":-))":"sehr_zufrieden",
    u":')":"Freudentränen",
    u"=L":"spielend",
    u";)":"Zwinker",
    u":-*":"Kuss",
    u":o":"schockiert",
    u":-(":"traurig",
    u"=D":"glücklich",
}
def convert_emoticons(text):
    for emoticon, description in EMOTICONS_DE.items():
        if emoticon in text:
            emojis.append(emoticon)
            text = text.replace(emoticon, description)
    return text

In [5]:
#find_maximum_depth(1051784054, 1)
start = time.time()
#m_depth = max([(posting_id,find_maximum_depth(posting_id, 1)) for posting_id in df['ID_Posting']],key=lambda x: x[1])
end = time.time()
#print('time: ', end-start)
#print('max depth: ',m_depth)

#time:  713.1474175453186
#max depth:  (1056921882, 15)

#start = time.time()
#delete_list = [find_maximum_depth(posting_id, 1)) for posting_id in df['ID_Posting']],key=lambda x: x[1])
#end = time.time()
#print('time: ', end-start)
#print('max depth: ',m_depth)

In [6]:
#emoticons preprocessing with translation
#emot_obj = emot.core.emot()
text = "Ich wills net zugebn :-) ;) :-( :-)))" 
#print(list(set(emojis)))
convert_emoticons(text)


'Ich wills net zugebn zufrieden Zwinker traurig zufrieden))'

In [7]:
# spell correction for austrian language
text = 'wos kann ich tun für dich ?'
spell = SpellChecker(language='de')
misspelled = spell.unknown(text.split())
corrected_text = text
for word in misspelled:
    corrected_text = corrected_text.replace(word, spell.correction(word))
    
print(corrected_text)

was kann ich tun für dich ?


In [8]:
# translate english to german
#translator= Translator(to_lang="German")
#translation = translator.translate(corrected_text)
#print(translation)

#print(tss.google(corrected_text, from_language='en', to_language='de'))

In [9]:
#def misspelling_correction(te)

In [10]:
# preprocessing pipeline
counter = 0
spell = SpellChecker(language='de')
def preprocessing(text):
    text = str(text)
    #transform emojis
    text = convert_emoticons(text)
    #spell correction
    misspelled = spell.unknown(text.split())
    corrected_text = text
    for word in misspelled:
        if spell.correction(word) is not None:
            corrected_text = corrected_text.replace(word, spell.correction(word))
    text = corrected_text
    
    
    #translate everything to german
    #text = tss.google(text, from_language='en', to_language='de')
    #lowercase-punctuation-stopwords
    tokens = nltk.word_tokenize(text)
    lowercase_tokens = [token.lower() for token in tokens]
    clean_tokens = [re.sub(r'[^\w\s]', '', token) for token in lowercase_tokens]
    stop_words = stopwords.words("german")
    filtered_words = [w for w in clean_tokens if w not in stop_words and w]
    #return filtered_words
    
    # Lemmatization
    nlp = spacy.load('de_core_news_md')
    filtered_lemma = []
    for w in filtered_words:
        result = ' '.join([token.lemma_ for token in nlp(w)]) 
        filtered_lemma.append(result)
        
    #clear_output()
    # increment the counter
    #global counter
    #counter += 1
    #print(counter)
    return ' '.join(filtered_lemma)
    
    
    

In [11]:
print('cpu pool ', mp.cpu_count())

from concurrent.futures import ThreadPoolExecutor

def apply_with_threads(df, func, n_threads=4):
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        df['p_merged_text'] = list(executor.map(func, df['merged_text']))
    return df

def apply_with_processes(df, func, n_processes=mp.cpu_count()):
    pool = mp.Pool(processes=n_processes)
    df['p_merged_text'] = pool.map(func, df['merged_text'])
    pool.close()
    pool.join()
    return df




cpu pool  40


In [12]:

df1 = df.copy()
df1['Headline'] = df1['Headline'].fillna('')
text_columns = ['Comment', 'Headline']
df1["merged_text"] = df1[text_columns].swifter.apply(lambda x: " | ".join(x), axis=1)

df1_sm = df1.iloc[:50000]

if __name__ == '__main__':
    print('hier')
    start = time.time()
    # create a multiprocess dataframe
    meta_df = pd.DataFrame({'ID_Posting': pd.Series(dtype='int64'),
                   'merged_text': pd.Series(dtype='str')})
    
    #ddf1 = dd.from_pandas(df1[['ID_Posting','merged_text']], npartitions=mp.cpu_count())
    ddf1_sm = dd.from_pandas(df1_sm[['ID_Posting','merged_text']], npartitions=mp.cpu_count())
    processed_df = ddf1_sm.map_partitions(lambda df: df.apply(lambda row: preprocessing(row['merged_text']),axis=1), meta=("p_merged_text", object)).compute(scheduler="processes")
    #df1['p_merged_text'] = df1['merged_text'].swifter.apply(lambda x: preprocessing(x))
    end = time.time()
    print('time: ', (end-start))
    
    
    
    




#df1['p_merged_textt'] = df1['merged_text'].apply(lambda x: preprocessing(x))



#all emojis in the text
#print(list(set(emojis)))
df1_sm["p_merged_text"] = processed_df
df1_sm.head()
#df1.head()

Pandas Apply:   0%|          | 0/421572 [00:00<?, ?it/s]

hier
time:  33700.4441716671


C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_36720\248835007.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sm["p_merged_text"] = processed_df


,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive,merged_text,p_merged_text
0,1049737336,-1,2000114365338,568012,1.0,,Das Virus ist wohl doch VIEL ansteckender als ...,2020-02-10 08:48:31.513,False,True,Das Virus ist wohl doch VIEL ansteckender als ...,Virus wohl ansteckend denken wohl wirklich Luf...
2,1049737620,1049737336,2000114365338,505220,1.0,,"So ist es, somit sind bestimmt auch deutlich m...",2020-02-10 08:59:34.357,False,True,"So ist es, somit sind bestimmt auch deutlich m...",somit bestimmen deutlich mehr Mensch China inf...
4,1049737708,-1,2000114365338,508661,1,,Das ist für mich eine Horrorvorstellung. Final...,2020-02-10 09:03:20.527,False,True,Das ist für mich eine Horrorvorstellung. Final...,Horrorvorstellung Final Destination x
6,1049737722,-1,2000114365338,583906,1,,"Wäre interessant zu wissen, ob vor allem schon...",2020-02-10 09:03:48.857,True,False,"Wäre interessant zu wissen, ob vor allem schon...",sein interessant wissen schon krank schwächen ...
8,1049737801,1049737746,2000114365338,635467,1,,Besser als Häfn! ;-),2020-02-10 09:07:10.360,False,True,Besser als Häfn! ;-) |,gut Häfn


In [13]:
#merging content with title and build tokens
df1_sm.drop('merged_text', axis=1, inplace=True)

df2 = df1_sm.copy()
#df2.rename(columns = {"p_comment":"comment","p_headline":"headline"}, inplace = True)
#text_columns = ["comment", "headline"]
#df2["merged_text"] = df2[text_columns].apply(lambda x: " | ".join(x), axis=1)
#df1["tokens"] = df1["merged_text"].map(lambda x: x.split())
df2.head()

C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_36720\3150821559.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sm.drop('merged_text', axis=1, inplace=True)


,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive,p_merged_text
0,1049737336,-1,2000114365338,568012,1.0,,Das Virus ist wohl doch VIEL ansteckender als ...,2020-02-10 08:48:31.513,False,True,Virus wohl ansteckend denken wohl wirklich Luf...
2,1049737620,1049737336,2000114365338,505220,1.0,,"So ist es, somit sind bestimmt auch deutlich m...",2020-02-10 08:59:34.357,False,True,somit bestimmen deutlich mehr Mensch China inf...
4,1049737708,-1,2000114365338,508661,1,,Das ist für mich eine Horrorvorstellung. Final...,2020-02-10 09:03:20.527,False,True,Horrorvorstellung Final Destination x
6,1049737722,-1,2000114365338,583906,1,,"Wäre interessant zu wissen, ob vor allem schon...",2020-02-10 09:03:48.857,True,False,sein interessant wissen schon krank schwächen ...
8,1049737801,1049737746,2000114365338,635467,1,,Besser als Häfn! ;-),2020-02-10 09:07:10.360,False,True,gut Häfn


In [14]:
##building phrases from all comments
tokens = df2["p_merged_text"].map(lambda x: x.split())
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tokens, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tokens], threshold=50)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


data_words = make_trigrams(tokens)
data_words_join = [" ".join(inner) for inner in data_words]
df2["p_merged_text"] = np.array(data_words_join)
#df2["tokens"] = np.array(data_words)
df2.head()

,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive,p_merged_text
0,1049737336,-1,2000114365338,568012,1.0,,Das Virus ist wohl doch VIEL ansteckender als ...,2020-02-10 08:48:31.513,False,True,Virus wohl ansteckend denken wohl wirklich Luf...
2,1049737620,1049737336,2000114365338,505220,1.0,,"So ist es, somit sind bestimmt auch deutlich m...",2020-02-10 08:59:34.357,False,True,somit bestimmen deutlich mehr Mensch China inf...
4,1049737708,-1,2000114365338,508661,1,,Das ist für mich eine Horrorvorstellung. Final...,2020-02-10 09:03:20.527,False,True,Horrorvorstellung Final Destination x
6,1049737722,-1,2000114365338,583906,1,,"Wäre interessant zu wissen, ob vor allem schon...",2020-02-10 09:03:48.857,True,False,sein interessant wissen schon krank schwächen ...
8,1049737801,1049737746,2000114365338,635467,1,,Besser als Häfn! ;-),2020-02-10 09:07:10.360,False,True,gut Häfn


In [15]:
#df2.to_csv('../data/p_comment.csv', encoding='utf-8', index=False)
df2.to_csv('../../data/p_comment_big_sm.csv', encoding='utf-8', index=False)